# JS ImageGrid from duckdb query

This notebook demonstrates the bare minimum pathway from:

S3 lance => duckdb => python => javascript image grid

Within the same notebook

MacOS fresh setup:

1. Install python: https://www.python.org/ftp/python/3.10.8/python-3.10.8-macos11.pkg

verify install in terminal:

```bash
➜ python3 --version
Python 3.10.8
```

2. Setup virtual environment and activate it:

```bash
python3 -m venv ~/.venv/lance

source ~/.venv/lance/bin/activate
```

for convience I include `source ~/.venv/lance/bin/activate` in my .zshrc, otherwise every new terminal session needs to run it.


2. Install prerequisites:

```bash
pip install --quiet pylance ipython jupyter ipython-sql SQLAlchemy duckdb-engine
```

3. Start a notebook server in the same directory as this notebook:

```bash
jupyter notebook
```

It should print out a link in the terminal. Open it in the browser.


4. Then open the notebook and you should be here

## Load the jupyter notebook extension 

In [1]:
%load_ext sql
%config SqlMagic.autopandas = True

## Connect to duckdb

In [2]:
%sql duckdb:///:memory:

TODO test extension. Supposedly this:
`--connection_arguments {"preload_extensions":["lance"],"config":{"allow_unsigned_extensions":true}}`
works

## Setup the data

In [3]:
import lance
uri = 's3://eto-public/datasets/oxford_pet/oxford_pet.lance'
oxford_pet = lance.dataset(uri)

# We can run sql on this directly via duckdb

In [4]:
%%sql df <<

SELECT filename, class, split, external_image
FROM oxford_pet
LIMIT 20;

 * duckdb:///:memory:
Done.
Returning data to local variable df


n.b.: the `df <<` assigns the output to a variable in the jupyter user namespace

In [5]:
df

In [6]:
old_prefix = 's3://eto-public/'
new_prefix = 'https://eto-public.s3.us-west-2.amazonaws.com/'
df['external_image'] = df.external_image.str.replace(old_prefix, new_prefix)

AttributeError: 'ResultSet' object has no attribute 'external_image'

We should do the above by default so we don't need to do pre-signed in demos

## Render this in html/js image grid

This basically consists of two components:

1. Use [_repr_html_](https://ipython.readthedocs.io/en/stable/config/integrating.html) to render HTML output below the cell
2. Instead of a backend server API, we use [IPython.notebook.kernel.execute](https://jakevdp.github.io/blog/2013/06/01/ipython-notebook-javascript-python-communication/)

### Copy/paste some html/css/js

Just for the example, safe to ignore

In [ ]:
STYLES = """
* {
  box-sizing: border-box;
}

body {
  margin: 0;
  font-family: Arial, Helvetica, sans-serif;
}

.header {
  text-align: center;
  padding: 32px;
}

.row {
  display: -ms-flexbox; /* IE 10 */
  display: flex;
  -ms-flex-wrap: wrap; /* IE 10 */
  flex-wrap: wrap;
  padding: 0 4px;
}

/* Create two equal columns that sits next to each other */
.column {
  -ms-flex: 50%; /* IE 10 */
  flex: 50%;
  padding: 0 4px;
}

.column img {
  margin-top: 8px;
  vertical-align: middle;
}

/* Style the buttons */
.btn {
  border: none;
  outline: none;
  padding: 10px 16px;
  background-color: #f1f1f1;
  cursor: pointer;
  font-size: 18px;
}

.btn:hover {
  background-color: #ddd;
}

.btn.active {
  background-color: #666;
  color: white;
}
"""

In [ ]:
DUMB_GRID="""
<!-- Header -->
        <div class="header" id="myHeader">
          <h1>Image Grid</h1>
          <p>Click on the buttons to change the grid view.</p>
          <button class="btn" onclick="one()">1</button>
          <button class="btn active" onclick="two()">2</button>
          <button class="btn" onclick="four()">4</button>
        </div>

        <!-- Photo Grid -->
        <div class="row"> 
          <div class="column">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
          </div>
          <div class="column">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
          </div>  
          <div class="column">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
          </div>
          <div class="column">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
            <img src="" style="width:100%">
          </div>
        </div>
"""

In [ ]:
GRID_JS = """

// Get the elements with class="column"
var elements = document.getElementsByClassName("column");

// Declare a loop variable
var i;

// Full-width images
function one() {
    for (i = 0; i < elements.length; i++) {
    elements[i].style.msFlex = "100%";  // IE10
    elements[i].style.flex = "100%";
  }
}

// Two images side by side
function two() {
  for (i = 0; i < elements.length; i++) {
    elements[i].style.msFlex = "50%";  // IE10
    elements[i].style.flex = "50%";
  }
}

// Four images side by side
function four() {
  for (i = 0; i < elements.length; i++) {
    elements[i].style.msFlex = "25%";  // IE10
    elements[i].style.flex = "25%";
  }
}

// Add active class to the current button (highlight it)
var header = document.getElementById("myHeader");
var btns = header.getElementsByClassName("btn");
for (var i = 0; i < btns.length; i++) {
  btns[i].addEventListener("click", function() {
    var current = document.getElementsByClassName("active");
    current[0].className = current[0].className.replace(" active", "");
    this.className += " active";
  });
}
"""

### Here's the magic
Use `IPython.notebook.kernel.execute` to let JS call back into python

In [ ]:
import json

# Imagine the %%sql magic returns an instance of this
class ResultSet:
    
    def __init__(self, df, user_ns_name):
        self.df = df
        self.user_ns_name = user_ns_name
    
    def to_json(self):
        return json.dumps(self.df.to_dict(orient='records'))
    
    def _repr_html_(self):
        return self._gen_html()
        
    def _gen_html(self):
        return f"""
        <!DOCTYPE html>
        <html>
          <head>
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <style>
              {STYLES}
            </style>
          </head>
        
          <body>
            {DUMB_GRID}
            <script>
            {self._gen_script()}
            </script>
          </body>
        </html>
        """

    def _gen_script(self):
        # This is like a simplified "/preview API call"
        python_cmd = f"{self.user_ns_name}.to_json()"
        callback = """
        {
            iopub: {
                // this is the response callback
                output: function(response) {
                
                    // Results come out as plain text
                    var output = response.content.data["text/plain"];
                    
                    // Remove unwanted characters that breaks json parsing
                    output = output.substring(1, output.length-1).replace("\\'","'");
                    
                    var rows = JSON.parse(output);

                    // Simple hack here just for illustration
                    var i = 0;
                    var columns = document.getElementsByClassName("column");
                    for (c of columns) {
                        var images = c.getElementsByTagName("img");  
                        for (img of images) {
                           img.setAttribute("src", rows[i]["external_image"]); 
                           i++; 
                        };
                    };
                }
            }
        }
        """
        
        config = """
        {
            silent: false,
            store_history: false,
            stop_on_error: true
        }
        """
        
        return f"""
        IPython.notebook.kernel.execute("{python_cmd}", {callback}, {config});
        {GRID_JS}
        """

In [ ]:
# The notebook magic function has access to the cell user namespace and can emit output variables 
# (this is how we saved the output to `df`)
# So inside the magic function, we can create an instance of ResultSet that knows it's own name

# so it will be equivalent to something like this:

rs = ResultSet(df, '__lance__sql__0')
__lance__sql__0 = rs

In [ ]:
__lance__sql__0

## Can we put it all together
If we hacked this into ipython-sql (or made our own), it would look like the following.
This won't work unless you installed chang's fork of ipython-sql

In [7]:
%load_ext sql
%config SqlMagic.autopandas = True
%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [8]:
import lance
uri = 's3://eto-public/datasets/oxford_pet/oxford_pet.lance'
oxford_pet = lance.dataset(uri)

In [18]:
%%sql --lance True hack <<

SELECT filename, class, split, object,
  replace(external_image, 's3://eto-public/', 'https://eto-public.s3.us-west-2.amazonaws.com/') as external_image
FROM oxford_pet
LIMIT 60;

 * duckdb:///:memory:
Done.
Returning data to local variable hack


In [20]:
hack.df

,filename,class,split,object,external_image
0,Abyssinian_100,Abyssinian,train,"{'name': ['cat'], 'pose': ['Frontal'], 'trunca...",https://eto-public.s3.us-west-2.amazonaws.com/...
1,Abyssinian_101,Abyssinian,train,"{'name': ['cat'], 'pose': ['Frontal'], 'trunca...",https://eto-public.s3.us-west-2.amazonaws.com/...
2,Abyssinian_102,Abyssinian,train,"{'name': ['cat'], 'pose': ['Frontal'], 'trunca...",https://eto-public.s3.us-west-2.amazonaws.com/...
3,Abyssinian_103,Abyssinian,train,"{'name': ['cat'], 'pose': ['Frontal'], 'trunca...",https://eto-public.s3.us-west-2.amazonaws.com/...
4,Abyssinian_104,Abyssinian,train,"{'name': None, 'pose': None, 'truncated': None...",https://eto-public.s3.us-west-2.amazonaws.com/...
5,Abyssinian_105,Abyssinian,train,"{'name': ['cat'], 'pose': ['Frontal'], 'trunca...",https://eto-public.s3.us-west-2.amazonaws.com/...
6,Abyssinian_106,Abyssinian,train,"{'name': ['cat'], 'pose': ['Frontal'], 'trunca...",https://eto-public.s3.us-west-2.amazonaws.com/...
7,Abyssinian_107,Abyssinian,train,"{'name': ['cat'], 'pose': ['Frontal'], 'trunca...",https://eto-public.s3.us-west-2.amazonaws.com/...
8,Abyssinian_108,Abyssinian,train,"{'name': ['cat'], 'pose': ['Frontal'], 'trunca...",https://eto-public.s3.us-west-2.amazonaws.com/...
9,Abyssinian_109,Abyssinian,train,"{'name': ['cat'], 'pose': ['Frontal'], 'trunca...",https://eto-public.s3.us-west-2.amazonaws.com/...


In [21]:
hack.df.to_json(r'df.json', orient="records")

In [23]:
import IPython
IPython.display.HTML(filename='./index.html')